# Data Cleaning


In [1]:
# Importing the libraries
import pandas as pd
import numpy as np

In [2]:
# import matplotlib
# matplotlib.use('agg')
# import matplotlib.pyplot as plt
# import seaborn as sns

In [19]:
file_path = '../../data/raw/challenge_campus_biomedico_2024.parquet'
df = pd.read_parquet(file_path, engine= 'pyarrow')

In [20]:
df_origin = df.copy()

In [7]:
# df = df_origin.copy

In [ ]:
df.info()

In [11]:
#count null values for each column
df.isnull().sum().sort_values(ascending=False)

data_disdetta                                460639
codice_provincia_erogazione                   28776
codice_provincia_residenza                    28380
ora_fine_erogazione                           28181
ora_inizio_erogazione                         28181
comune_residenza                                135
id_prenotazione                                   0
id_paziente                                       0
data_nascita                                      0
provincia_residenza                               0
codice_asl_residenza                              0
codice_comune_residenza                           0
tipologia_servizio                                0
codice_regione_residenza                          0
asl_residenza                                     0
sesso                                             0
regione_residenza                                 0
regione_erogazione                                0
data_contatto                                     0
codice_descr

The function __remove_disdette__ removes the rows of a DataFrame where the data_disdetta column is not null. This is useful for filtering the data, keeping only the rows that do not have an associated cancellation date.

In [21]:
df['data_disdetta'].isnull().value_counts()

data_disdetta
True     460639
False     23652
Name: count, dtype: int64

In [22]:
def remove_disdette(df) -> pd.DataFrame: 
    # Remove rows where 'data_disdetta' is not null
    df = df[df['data_disdetta'].isnull()]
    # # Drop columns with more than 50% missing values
    # df = df.loc[:, df.isnull().mean() < 0.5]
    return df


In [23]:
df = remove_disdette(df)
df['data_disdetta'].isnull().value_counts()

data_disdetta
True    460639
Name: count, dtype: int64

In [46]:
df.columns

Index(['id_prenotazione', 'id_paziente', 'data_nascita', 'sesso',
       'regione_residenza', 'codice_regione_residenza', 'asl_residenza',
       'codice_asl_residenza', 'provincia_residenza',
       'codice_provincia_residenza', 'comune_residenza',
       'codice_comune_residenza', 'tipologia_servizio', 'descrizione_attivita',
       'codice_descrizione_attivita', 'data_contatto', 'regione_erogazione',
       'codice_regione_erogazione', 'asl_erogazione', 'codice_asl_erogazione',
       'provincia_erogazione', 'codice_provincia_erogazione',
       'struttura_erogazione', 'codice_struttura_erogazione',
       'tipologia_struttura_erogazione',
       'codice_tipologia_struttura_erogazione', 'id_professionista_sanitario',
       'tipologia_professionista_sanitario',
       'codice_tipologia_professionista_sanitario', 'data_erogazione',
       'ora_inizio_erogazione', 'ora_fine_erogazione', 'data_disdetta'],
      dtype='object')

The method __identify_and_remove_outliers_zscore__ uses the z-score method to identify and remove outliers from a DataFrame. The z-score measures the distance of a value from the mean in terms of standard deviations. This method normalizes the data and considers values that deviate from the mean beyond a specified threshold (default is 3) as outliers. The outliers are removed for each specified column, returning a DataFrame without these anomalous values.

In [47]:
def identify_and_remove_outliers_zscore(df, columns, threshold=3):
    """
    Identifies and removes outliers using the z-score method (normalization).
    
    :param df: The original DataFrame.
    :param columns: The columns on which to apply outliers removal.
    :param threshold: The z-score threshold for outlier detection (default: 3).
    :return: A DataFrame with no outliers.
    """
    for col in columns:
        z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
        df = df[z_scores <= threshold]
    return df


In [48]:
# df = identify_and_remove_outliers_zscore(df, ['ora_inizio_erogazione', 'ora_fine_erogazione'])
# df['ora_inizio_erogazione', 'ora_fine_erogazione'].describe()

In [49]:
# df['ora_inizio_erogazione']

The function __smooth_noisy_data__ applies a moving average to smooth noisy data in a specified column of a DataFrame. Using a defined window size, the function calculates the average of the values within this window, thereby reducing fluctuations and noise in the data. This method is useful for obtaining a clearer representation of trends in the data.

In [50]:
def smooth_noisy_data(df, column, window_size=3):
  """
  Smooth noisy data using moving average.

  Args:
    df: The original DataFrame.
    column: The column to apply smoothing to.
    window_size: The size of the moving average window.

  Returns:
    A DataFrame with the smoothed data.
  """

  df[column] = df[column].rolling(window=window_size, min_periods=1).mean()
  return df


The function __remove_duplicates__ removes duplicates from a DataFrame. Using the drop_duplicates method from pandas, the function eliminates duplicate rows.

In [24]:
def remove_duplicates(df) -> pd.DataFrame:
    """
    Removes duplicates from dataset df.
    :param df:
    :return:
    """
    df.drop_duplicates(inplace=True)
    return df

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 460639 entries, 0 to 484290
Data columns (total 33 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   id_prenotazione                            460639 non-null  object 
 1   id_paziente                                460639 non-null  object 
 2   data_nascita                               460639 non-null  object 
 3   sesso                                      460639 non-null  object 
 4   regione_residenza                          460639 non-null  object 
 5   codice_regione_residenza                   460639 non-null  int64  
 6   asl_residenza                              460639 non-null  object 
 7   codice_asl_residenza                       460639 non-null  int64  
 8   provincia_residenza                        460639 non-null  object 
 9   codice_provincia_residenza                 433623 non-null  object 
 10  comune_reside

In [8]:
df.comune_residenza.isnull().value_counts()

comune_residenza
False    484156
True        135
Name: count, dtype: int64

The function __imputate_comune_residenza__ imputes missing values in the comune_residenza column of a DataFrame using ISTAT codes. It loads a dataset containing ISTAT codes and the names of Italian municipalities, then merges this dataset with the original DataFrame based on the municipality code. Finally, it renames the column with the municipality name and removes the excess columns, returning a DataFrame with the imputed values.

In [9]:
def imputate_comune_residenza(df):
    """
    Imputes missing values for 'comune_residenza' using ISTAT codes.

    Args:
        df: The DataFrame containing the data.

    Returns:
        The DataFrame with imputed values.
    """

    # Load ISTAT data
    istat_data = pd.read_excel('../../data/raw/Codici-statistici-e-denominazioni-al-30_06_2024.xlsx')
    
    # Create the mapping dictionary
    codice_comune_to_nome = pd.Series(istat_data['Denominazione in italiano'].values,
                                      index=istat_data['Codice Comune formato alfanumerico'])


    # Merge DataFrames on 'codice_comune_residenza'
    #df = pd.merge(df, istat_data, left_on='codice_comune_residenza', right_on='Codice Comune formato alfanumerico', how='left')

    df['comune_residenza'].fillna(df['codice_comune_residenza'].map(codice_comune_to_nome), inplace=True)

    # df.drop('comune_residenza', axis=1, inplace=True)

    # Rename the column and remove the excess column (if necessary)
    #df.rename(columns={'Denominazione in italiano': 'comune_residenza'}, inplace=True)
    #df.drop('Codice Comune formato alfanumerico', axis=1, inplace=True)
    
    return df, codice_comune_to_nome

In [10]:
df , codice_comune_to_nome = imputate_comune_residenza(df)

df.comune_residenza.isnull().value_counts()

/tmp/ipykernel_2809494/1826514609.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['comune_residenza'].fillna(df['codice_comune_residenza'].map(codice_comune_to_nome), inplace=True)


comune_residenza
False    484156
True        135
Name: count, dtype: int64

In [24]:
df[df.comune_residenza.isnull()].codice_comune_residenza.value_counts()

codice_comune_residenza
1168    135
Name: count, dtype: int64

In [10]:
istat_data = pd.read_excel('../../data/raw/Codici-statistici-e-denominazioni-al-30_06_2024.xlsx')

In [12]:
app = pd.Series(istat_data['Denominazione in italiano'].values,
                index=istat_data['Codice Comune formato alfanumerico'])

In [21]:
app[app.isnull()== True]

Codice Comune formato alfanumerico
1168    NaN
dtype: object

In [7]:
df.codice_comune_residenza

0          4089
1         48005
2         37006
3         63049
4         68028
          ...  
484286    78081
484287    12123
484288    71024
484289    63049
484290    80092
Name: codice_comune_residenza, Length: 484291, dtype: int64

The function __fill_missing_comune_residenza__ is designed to fill missing values in the comune_residenza column of a DataFrame using a provided mapping dictionary. This dictionary maps municipality codes to municipality names. The function also handles a special case where the municipality code '1168' is replaced with 'None' (representing Turin). The missing values in the comune_residenza column are then filled using the mapping dictionary. The function returns the DataFrame with the filled values

In [53]:
def fill_missing_comune_residenza(df, codice_comune_to_nome):
      """
      Fills missing values in the 'comune_residenza' column using a mapping.
    
      Args:
        df: The DataFrame containing the data.
        codice_comune_to_nome: A dictionary mapping the municipality code to the municipality name.
    
      Returns:
        The DataFrame with filled missing values.
      """
    
      # Handle the special case: municipality of None (Turin)
      df['codice_comune_residenza'] = df['codice_comune_residenza'].replace('1168', 'None')
    
      # Fill missing values using the mapping
      df['comune_residenza'] = df['comune_residenza'].fillna(df['codice_comune_residenza'].map(codice_comune_to_nome))

      return df

The function __check_missing_values_same_row__ is designed to identify and count the rows in a DataFrame where both ora_inizio_erogazione and ora_fine_erogazione columns have missing values. It checks for missing values in these two columns simultaneously and prints the number of rows where both columns are missing. 

In [54]:
def check_missing_values_same_row(df):
    """
    Checks if missing values in 'ora_inizio_erogazione' and 'ora_fine_erogazione' are in the same rows.

    Args:
        df: The DataFrame to check.

    Returns:
        None
    """

    missing_both = df[['ora_inizio_erogazione', 'ora_fine_erogazione']].isna().all(axis=1)
    num_rows_with_both_missing = missing_both.sum()
    print(f"Number of rows with both 'ora_inizio_erogazione' and 'ora_fine_erogazione' missing: {num_rows_with_both_missing}")


In [55]:
# menage missing start values
df['ora_inizio_erogazione'].isnull().value_counts()

ora_inizio_erogazione
False    456110
True       4529
Name: count, dtype: int64

In [56]:
#count missing values for ora_inizio_erogazione
def check_missing_values_start(df):
    null_values = df['ora_inizio_erogazione'].isnull().sum()
    print(f"Number of missing values in 'ora_inizio_erogazione': {null_values}")

In [57]:
# count missing values for ora_fine_erogazione
def check_missing_values_end(df):
    null_values = df['ora_fine_erogazione'].isnull().sum()
    print(f"Number of missing values in 'ora_fine_erogazione': {null_values}")

In [58]:
check_missing_values_start(df)
check_missing_values_end(df)

Number of missing values in 'ora_inizio_erogazione': 4529
Number of missing values in 'ora_fine_erogazione': 4529


In [59]:
def impute_ora_inizio_and_fine_erogazione(df:pd.DataFrame) -> pd.DataFrame:
    
    # Convert 'ora_inizio_erogazione' and 'ora_fine_erogazione' to datetime
    df['ora_inizio_erogazione'] = pd.to_datetime(df['ora_inizio_erogazione'], utc=True, errors='coerce')
    df['ora_fine_erogazione'] = pd.to_datetime(df['ora_fine_erogazione'], utc=True, errors='coerce')

    df_non_missing_values = df.dropna(subset=['ora_inizio_erogazione', 'ora_fine_erogazione']).copy()
    df_non_missing_values['duration'] = (df['ora_fine_erogazione'] - df['ora_inizio_erogazione']).dt.total_seconds()

    mean_duration_by_attivita = df_non_missing_values.groupby('codice_descrizione_attivita')['duration'].mean()
    mean_duration = pd.to_timedelta(mean_duration_by_attivita, unit='s')
    # print(mean_duration_by_attivita)

    # Convert series to dictionary
    mean_duration_dict = mean_duration.to_dict()
    # return mean_duration_dict


    for index, row in df.iterrows():
        if pd.isnull(row['ora_inizio_erogazione']) and pd.isnull(row['ora_fine_erogazione']) and pd.isnull(row['data_disdetta']):
            codice_attivita = row['codice_descrizione_attivita']
            
            if codice_attivita in mean_duration_dict:
                durata_media = mean_duration_dict[codice_attivita]
                data_erogazione = pd.to_datetime(row['data_erogazione'], utc=True)
                df.at[index, 'ora_inizio_erogazione'] = data_erogazione.strftime('%Y-%m-%d %H:%M:%S%z')
                df.at[index, 'ora_fine_erogazione'] = (data_erogazione + durata_media).strftime('%Y-%m-%d %H:%M:%S%z')
        

In [60]:
impute_ora_inizio_and_fine_erogazione(df)

In [61]:
df.isnull().sum().sort_values(ascending=False)

data_disdetta                                460639
codice_provincia_erogazione                   27396
codice_provincia_residenza                    27016
comune_residenza                                130
id_prenotazione                                   0
codice_regione_residenza                          0
asl_residenza                                     0
id_paziente                                       0
data_nascita                                      0
provincia_residenza                               0
codice_asl_residenza                              0
codice_comune_residenza                           0
tipologia_servizio                                0
descrizione_attivita                              0
codice_descrizione_attivita                       0
sesso                                             0
regione_residenza                                 0
regione_erogazione                                0
data_contatto                                     0
codice_asl_e

In [63]:
df.to_parquet('../../data/processed/challenge_campus_biomedico_2024_cleaned_v1.parquet')

In [ ]:
# Apply the function to the DataFrame
df, codice_comune_to_nome = imputate_comune_residenza(df)
df = fill_missing_comune_residenza(df, codice_comune_to_nome)
df = impute_ora_inizio_and_fine_erogazione(df)   
df = remove_disdette(df)
df = identify_and_remove_outliers_zscore(df, ['ora_inizio_erogazione', 'ora_fine_erogazione'])
df = smooth_noisy_data(df, 'ora_inizio_erogazione')
df = remove_duplicates(df)
